In [0]:
import os
import numpy
import matplotlib.pyplot as pyplot
from itertools import groupby 
import random
import shutil
%matplotlib inline

In [2]:
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [3]:
!wget pages.mini.pw.edu.pl/~gorzynskik/obrazki/dataset-v3.zip

--2020-05-12 20:18:07--  http://pages.mini.pw.edu.pl/~gorzynskik/obrazki/dataset-v3.zip
Resolving pages.mini.pw.edu.pl (pages.mini.pw.edu.pl)... 194.29.178.29
Connecting to pages.mini.pw.edu.pl (pages.mini.pw.edu.pl)|194.29.178.29|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1030046 (1006K) [application/zip]
Saving to: ‘dataset-v3.zip’

dataset-v3.zip      100%[===================>]   1006K  1.40MB/s    in 0.7s    

2020-05-12 20:18:08 (1.40 MB/s) - ‘dataset-v3.zip’ saved [1030046/1030046]



In [0]:
shutil.unpack_archive('dataset-v3.zip')

In [0]:
!rm -r data

In [0]:
def structureData(source_data, train_perc):
    images = []

    for (dirpath, dirnames, filenames) in os.walk(source_data):
        images.extend(filenames)

    images = [n for n in images if n.endswith('.png')]

    data_path = './data'
    if not os.path.exists(data_path):
        os.makedirs(data_path)

    util_func = lambda x: x[0] 
    temp = sorted(images, key = util_func) 
    res = [list(ele) for i, ele in groupby(temp, util_func)]

    train_path = os.path.join(data_path, 'train')
    os.makedirs(train_path)
    test_path = os.path.join(data_path, 'test')
    os.makedirs(test_path)

    for i in range(len(res)):
        perc = 0.7
        z_0 = res[i]
        train_len = int(len(z_0) * 0.7)
        random.shuffle(z_0)
        train_p = z_0[:train_len]
        test_p = z_0[train_len:]
        tr_path = os.path.join(train_path, str(i))
        os.makedirs(tr_path)
        te_path = os.path.join(test_path, str(i))
        os.makedirs(te_path)
        for name in train_p:
            shutil.copy(os.path.join(source_data, name), tr_path)
        for name in test_p:
            shutil.copy(os.path.join(source_data, name), te_path)

In [0]:
structureData('./dataset', 0.7)

In [0]:
datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        rotation_range=10,
        horizontal_flip=True,
        vertical_flip=False)


In [0]:
img_shape = (28, 28)

In [13]:
train_generator = datagen.flow_from_directory(
    'data/train',
    target_size=img_shape,
    color_mode = 'grayscale',
    batch_size=32,
    class_mode='categorical')

Found 472 images belonging to 7 classes.


In [14]:
test_generator = datagen.flow_from_directory(
    'data/test',
    target_size= img_shape,
    color_mode = 'grayscale',
    batch_size=32,
    class_mode='categorical')

Found 206 images belonging to 7 classes.


In [0]:
input_shape = (img_shape[0], img_shape[1], 1)

In [16]:
model = Sequential()
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = input_shape))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 7 , activation = 'softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 75)        750       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 75)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 50)        33800     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 50)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 25)          11275     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 25)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)              

In [17]:
model.fit_generator(
    train_generator,
    steps_per_epoch=32,
    epochs=20,
    validation_data=test_generator)

Epoch 1/20
32/32 [==============================] - 4s 132ms/step - loss: 1.6730 - accuracy: 0.3135 - val_loss: 1.2964 - val_accuracy: 0.5146
Epoch 2/20
32/32 [==============================] - 4s 112ms/step - loss: 1.1332 - accuracy: 0.5774 - val_loss: 0.6864 - val_accuracy: 0.6748
Epoch 3/20
32/32 [==============================] - 4s 110ms/step - loss: 0.7928 - accuracy: 0.7282 - val_loss: 0.5761 - val_accuracy: 0.7427
Epoch 4/20
32/32 [==============================] - 3s 109ms/step - loss: 0.5587 - accuracy: 0.8230 - val_loss: 0.5969 - val_accuracy: 0.8301
Epoch 5/20
32/32 [==============================] - 3s 109ms/step - loss: 0.4146 - accuracy: 0.8562 - val_loss: 0.4305 - val_accuracy: 0.8495
Epoch 6/20
32/32 [==============================] - 4s 110ms/step - loss: 0.3649 - accuracy: 0.8601 - val_loss: 0.4942 - val_accuracy: 0.8495
Epoch 7/20
32/32 [==============================] - 3s 109ms/step - loss: 0.3186 - accuracy: 0.8879 - val_loss: 0.1915 - val_accuracy: 0.9029
Epoch 

In [0]:
model.save("model_v1.h5")